# Robust parsing of unrecignized code snippets
Doing this in an extra notebook in order to be able to import the code table from the db and to not have to re-run all prompts (expensive)

In [12]:
import os
import pandas as pd
import sqlite3

db_path = "../../data/giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
code = pd.read_sql("SELECT * FROM translated_scratch_prompts_with_code", conn)
code

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,model_version,programming_language
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,To use **Dummy Hot Encoding** on the `'embarke...,chatgpt-4o-latest,python
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,To encode and compress a JavaScript object int...,chatgpt-4o-latest,javascript
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,Certainly! You can extract the `persona` colum...,chatgpt-4o-latest,python
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,That’s a great idea — and quite a common pain ...,chatgpt-4o-latest,python
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Certainly! You can use the Blender API (`bpy`)...,chatgpt-4o-latest,python
...,...,...,...,...,...,...,...,...,...
199,200,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies the `parse_obj` fu...,claude-sonnet-4-20250514,python
200,201,72,1534,can you write me some python code to count the...,Woman (cisgender),11,```python\nimport json\n\ndef count_objects_in...,claude-sonnet-4-20250514,python
201,202,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the two dataframes using the 'cl...,claude-sonnet-4-20250514,python
202,203,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,Use the **Elbow Method** with Within-Cluster S...,claude-sonnet-4-20250514,python


# Extract code snippets

In [18]:
import re

def parse_code_snippets(text):
    """
    Extract code snippets marked with triple backticks and any of these language tags:
    python, javascript, js, typescript, ts.
    Joins all found code into a single string separated by newlines.
    """
    pattern = r"```(?:python|javascript|js|typescript|ts|html|css)[ \t]*\n(.*?)(?=```)"
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    code_snippets = [m.strip() for m in matches]
    return "\n".join(code_snippets)



columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]

for column in columns:
    code['code'] = code['model_reply'].apply(parse_code_snippets)

code

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,model_version,programming_language,code
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,To use **Dummy Hot Encoding** on the `'embarke...,chatgpt-4o-latest,python,import pandas as pd\nfrom sklearn.preprocessin...
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,To encode and compress a JavaScript object int...,chatgpt-4o-latest,javascript,"import pako from ""pako"";\n\nfunction encodeObj..."
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,Certainly! You can extract the `persona` colum...,chatgpt-4o-latest,python,import pandas as pd\n\n# Sample data\ndf = pd....
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,That’s a great idea — and quite a common pain ...,chatgpt-4o-latest,python,"import fitz # PyMuPDF\n doc = fitz.open(""p..."
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Certainly! You can use the Blender API (`bpy`)...,chatgpt-4o-latest,python,import bpy\nimport mathutils\n\n# === CONFIGUR...
...,...,...,...,...,...,...,...,...,...,...
199,200,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies the `parse_obj` fu...,claude-sonnet-4-20250514,python,def test_odoo_ticket_parse_obj():\n raw_tic...
200,201,72,1534,can you write me some python code to count the...,Woman (cisgender),11,```python\nimport json\n\ndef count_objects_in...,claude-sonnet-4-20250514,python,import json\n\ndef count_objects_in_json(json_...
201,202,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the two dataframes using the 'cl...,claude-sonnet-4-20250514,python,"merged_df = pd.merge(df_train_claim, df_train_..."
202,203,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,Use the **Elbow Method** with Within-Cluster S...,claude-sonnet-4-20250514,python,


## Extract other snippets using gpt 4o

In [19]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate



def parse_reply(reply):

    load_dotenv()

    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    openai_model = "gpt-4o"
    llm = ChatOpenAI(temperature=0.0, model=openai_model)

    class OutputFormat(BaseModel):
        code: str = Field(description="The code contained in the model reply")

    structured_llm = llm.with_structured_output(OutputFormat)

    system_prompt = SystemMessagePromptTemplate.from_template(
        "You are an AI assistant that helps parse llm replies."
    )

    user_prompt = HumanMessagePromptTemplate.from_template(
        """You are tasked with extracting code from  lmm replies.
                    The reply is here for you to examine
                    ---
                    {reply}
                    ---

                   The code might be spread out into multiple paragraphs. Concatenate code that belongs together and output it to the 'code' field.""",
                        input_variables=["reply"]
                )

    complete_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
    print(f"parsing next reply")
    chain_one = (
            {"reply": lambda x: x["reply"]}
            | complete_prompt
            | structured_llm
            | {"code": lambda x: x.code,}
    )

    return chain_one.invoke({"reply": reply})["code"]


# Only apply to rows where code is NaN or empty string
mask = code['code'].isna() | (code['code'].str.strip() == "")
code.loc[mask, 'code'] = code.loc[mask, 'model_reply'].apply(parse_reply)
code


parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply
parsing next reply


,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,model_version,programming_language,code
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,To use **Dummy Hot Encoding** on the `'embarke...,chatgpt-4o-latest,python,import pandas as pd\nfrom sklearn.preprocessin...
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,To encode and compress a JavaScript object int...,chatgpt-4o-latest,javascript,"import pako from ""pako"";\n\nfunction encodeObj..."
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,Certainly! You can extract the `persona` colum...,chatgpt-4o-latest,python,import pandas as pd\n\n# Sample data\ndf = pd....
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,That’s a great idea — and quite a common pain ...,chatgpt-4o-latest,python,"import fitz # PyMuPDF\n doc = fitz.open(""p..."
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Certainly! You can use the Blender API (`bpy`)...,chatgpt-4o-latest,python,import bpy\nimport mathutils\n\n# === CONFIGUR...
...,...,...,...,...,...,...,...,...,...,...
199,200,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies the `parse_obj` fu...,claude-sonnet-4-20250514,python,def test_odoo_ticket_parse_obj():\n raw_tic...
200,201,72,1534,can you write me some python code to count the...,Woman (cisgender),11,```python\nimport json\n\ndef count_objects_in...,claude-sonnet-4-20250514,python,import json\n\ndef count_objects_in_json(json_...
201,202,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the two dataframes using the 'cl...,claude-sonnet-4-20250514,python,"merged_df = pd.merge(df_train_claim, df_train_..."
202,203,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,Use the **Elbow Method** with Within-Cluster S...,claude-sonnet-4-20250514,python,# Import necessary libraries\nimport matplotli...


In [20]:
code.to_sql("translated_scratch_prompts_with_code", conn, if_exists="replace", index=False)

204